In [1]:
import os
import numpy
from pyspark.sql.functions import *
from pyspark.sql.types import DoubleType
spark_home = os.environ.get('SPARK_HOME', None)
# review = spark.read.json("Yelp/review.json")


In [2]:
import plotly
plotly.tools.set_credentials_file(username='amcire96', api_key='sej35ud4YbSOfIshhhZg')
# print(plotly.__version__)

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.figure_factory as FF
import requests
requests.packages.urllib3.disable_warnings()



In [3]:
review = spark.read.json("/user/hduser1/Yelp/review.json")
business = spark.read.json("/user/hduser1/Yelp/business.json")
checkin = spark.read.json("/user/hduser1/Yelp/checkin.json")
tip = spark.read.json("/user/hduser1/Yelp/tip.json")
user = spark.read.json("/user/hduser1/Yelp/user.json")

print("BUSINESS")
business.printSchema()
print()
print("CHECKIN")
checkin.printSchema()
print()
print("REVIEW")
review.printSchema()
print()
print("TIP")
tip.printSchema()
print()
print("USER")
user.printSchema()

BUSINESS
root
 |-- address: string (nullable = true)
 |-- attributes: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- business_id: string (nullable = true)
 |-- categories: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- city: string (nullable = true)
 |-- hours: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- is_open: long (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: long (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- type: string (nullable = true)

()
CHECKIN
root
 |-- business_id: string (nullable = true)
 |-- time: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- type: string (nullable = true)

()
REVIEW
root
 |-- business_id: 

In [3]:
print(review.select("type").head(10))

[Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review'), Row(type=u'review')]


In [4]:
extreme_reviews = review.where(review.stars % 4 == 1).repartition(300)
print(extreme_reviews.count())
print(extreme_reviews.where(extreme_reviews.stars == 1).count())
print(extreme_reviews.where(extreme_reviews.stars == 5).count())
print(extreme_reviews.select("stars").head(50))
print(extreme_reviews.head(5))

2244577
540377
1704200
[Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=1), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=1), Row(stars=5), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=1), Row(stars=1), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5), Row(stars=5)]
[Row(business_id=u'7u_1pr2Q8ma3mAYR_ZzBjg', cool=0, date=u'2015-07-10', funny=0, review_id=u'8j4_uvubhLH6Qz9Hm1x-Sw', stars=1, text=u'Dishwasher Inconvenience. I had been a Best Buy appliance loyalist until now. Granted, hadn\'t bought any for maybe 8 years but they are who I

In [5]:

extreme_reviews = extreme_reviews.withColumn("stars", extreme_reviews["stars"].cast(DoubleType()))

In [6]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler, Binarizer
from pyspark.ml import Pipeline
from pyspark.ml.linalg import Vectors



assembler = VectorAssembler(inputCols=["useful", "cool", "funny"], outputCol="features")
label_binarizer = Binarizer(threshold=3.0, inputCol="stars", outputCol="label")
logistic_reg = LogisticRegression().setFeaturesCol("features").setPredictionCol("prediction")

pipeline = Pipeline(stages=[assembler, label_binarizer, logistic_reg])


model = pipeline.fit(extreme_reviews)

predictions = model.transform(extreme_reviews)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(accuracy)


0.781907236865


In [7]:
from pyspark.ml.feature import Tokenizer, CountVectorizer

tokenizer = Tokenizer(inputCol="text", outputCol="words")
cv = CountVectorizer(minTF=100.0, minDF=10.0, inputCol="words", outputCol="features")
label_binarizer = Binarizer(threshold=3.0, inputCol="stars", outputCol="label")
logistic_reg = LogisticRegression().setFeaturesCol("features").setPredictionCol("prediction")

pipeline = Pipeline(stages=[tokenizer, cv, label_binarizer, logistic_reg])

model = pipeline.fit(extreme_reviews)

predictions = model.transform(extreme_reviews)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(accuracy)




0.759250406647


In [ ]:
from pyspark.ml.feature import Word2Vec

tokenizer = Tokenizer(inputCol="text", outputCol="words")
word2Vec = Word2Vec(vectorSize=500, minCount=100, inputCol="words", outputCol="features")
label_binarizer = Binarizer(threshold=3.0, inputCol="stars", outputCol="label")
logistic_reg = LogisticRegression().setFeaturesCol("features").setPredictionCol("prediction")

pipeline = Pipeline(stages=[tokenizer, word2Vec, label_binarizer, logistic_reg])

model = pipeline.fit(extreme_reviews)

predictions = model.transform(extreme_reviews)

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)

print(accuracy)


In [4]:
business = business.repartition(300)
user = user.repartition(300)
review = review.repartition(300)

In [32]:
b = business.select("city").distinct().toPandas()
print(b)

                         city
0                 Middlefield
1    Saint-Marc-sur-Richelieu
2     Sainte-Anne-des-Plaines
3                       Ratho
4                Mississauaga
5                       Tempe
6                Sainte-Adele
7                  Harrisburg
8             North Las Vegas
9                      Auburn
10                Wilkinsburg
11               Saint-Hubert
12                Indian Hook
13                    Phoenix
14                 Georgetown
15               West Mifflin
16                  Kirkcaldy
17                      Savoy
18             Upper St Clair
19        Pointe-Aux-Trembles
20             South Mountain
21     Bonnyrigg and Lasswade
22                  Brunswick
23                   Rosemère
24            South Las Vegas
25                  Tottenham
26                   Ben Avon
27                    Baldwin
28                     frazer
29                  Wickliffe
..                        ...
848              Kornwestheim
849       

In [40]:
print(b.sort("city"))
print(len(b))
print(len(b.drop_duplicates()))

878
878


In [5]:
# filter users with less than 5 reviews
valid_users = user.alias('u').join(review.alias('r'), user.user_id == review.user_id).select("u.user_id").groupby("user_id").count()
# print(len(valid_users.collect()))
valid_users = valid_users.where("count >= 5")



In [8]:
valid_users_review_count_df = valid_users.select("count").toPandas()['count']


In [15]:
valid_users_review_count_data = Data([Histogram(x=valid_users_review_count_df)])
py.iplot(valid_users_review_count_data)

/usr/local/lib/python2.7/dist-packages/plotly/plotly/plotly.py:215: UserWarning:

Woah there! Look at all those points! Due to browser limitations, the Plotly SVG drawing functions have a hard time graphing more than 500k data points for line charts, or 40k points for other types of charts. Here are some suggestions:
(1) Use the `plotly.graph_objs.Scattergl` trace object to generate a WebGl graph.
(2) Trying using the image API to return an image instead of a graph URL
(3) Use matplotlib
(4) See if you can create your visualization with fewer data points

If the visualization you're using aggregates points (e.g., box plot, histogram, etc.) you can disregard this warning.



In [6]:
user_geo = (business.alias('b').join(
            review.alias('r'),business.business_id == review.business_id).join(
            valid_users.alias('u'), review.user_id == valid_users.user_id).select(
            ['u.user_id', 'b.city']))

# user_geo.printSchema()
# print(user_geo.head(10))

In [7]:
user_geo_count = user_geo.groupby(["user_id", "city"]).count().orderBy(["user_id", "count"], ascending=[1, 0])

# user_geo_count.printSchema()
# print(user_geo_count.head(10))

In [8]:
user_num_cities = user_geo_count.groupby("user_id").count().orderBy("count", ascending=[0])

# print(user_num_cities.head(10))
# print(user_num_cities.orderBy("count", ascending=[1]).head(10))

In [12]:
user_num_cities_df = user_num_cities.select("count").toPandas()['count']


In [13]:
print(user_num_cities_df.head(10))
print(len(user_num_cities_df))

0    74
1    58
2    52
3    50
4    49
5    47
6    47
7    47
8    47
9    46
Name: count, dtype: int64
179097


In [15]:
num_cities_data = Data([Histogram(x=user_num_cities_df)])
py.iplot(num_cities_data)

# group_labels = ['cities a user has reviewed in']
# fig = FF.create_distplot([user_num_cities_df], group_labels)
# py.iplot(fig)


In [9]:
user_frac_cities = (user_geo_count.alias("ugc").join(valid_users.alias("vu"), col("ugc.user_id") == col("vu.user_id")).select(
                    [col("ugc.user_id"), col("ugc.city"), (col("ugc.count") / col("vu.count")).alias("frac")]).orderBy("frac", ascending=[1]))
user_frac_cities = user_frac_cities.groupby("user_id").agg(max("frac").alias("maxFrac"))

# user_frac_cities.printSchema()
# user_frac_cities.head(5)

In [17]:
user_frac_cities_df = user_frac_cities.select("maxFrac").toPandas()['maxFrac']

In [19]:
frac_cities_data = Data([Histogram(x=user_frac_cities_df)])
py.iplot(frac_cities_data)

In [10]:
valid_user_frac_cities = (user_geo_count.alias("ugc").join(valid_users.alias("vu"), col("ugc.user_id") == col("vu.user_id")).select(
                        [col("ugc.user_id"), col("ugc.city"), (col("ugc.count") / col("vu.count")).alias("frac")]).where("frac > 0.1"))
valid_user_and_city = valid_user_frac_cities.select([col("user_id"), col("city")])

# print(valid_user_and_city.select("city").collect())



In [11]:
users_in_both = (valid_user_and_city.alias("df1").join(valid_user_and_city.alias("df2"), col("df1.city") < col("df2.city"))
                 .join(user.select("user_id").alias("u"), (col("u.user_id") == col("df1.user_id")) & (col("u.user_id") == col("df2.user_id")))
                 .groupby([col("df1.city").alias("city1"), col("df2.city").alias("city2")]).count()
                 .select([col("city1"), col("city2"), col("count").alias("users_in_both")]).where("users_in_both >= 10"))

# print(users_in_both.toPandas())

In [33]:
city_users = valid_user_and_city.groupby("city").count().select([col("city"), (col("count").alias("city_users"))]).where("city_users>=10")
city_users_df = city_users.toPandas()

print(city_users_df)

                    city  city_users
0            Middlefield          39
1                  Tempe       18761
2             Harrisburg         172
3        North Las Vegas        4473
4           Saint-Hubert          10
5                Phoenix       46252
6             Georgetown          35
7                  Savoy         128
8           West Mifflin         157
9              Brunswick         199
10       South Las Vegas          41
11             Fitchburg         476
12             Newmarket         299
13             Wickliffe          58
14             King City          11
15   Lower Lawrenceville          40
16             Tallmadge          79
17                Bolton          22
18              Hinckley          11
19              Paradise         105
20            Woodbridge         261
21             Glenndale          33
22            Bainbridge          14
23             Avon Lake         199
24            McKeesport          45
25             Inverness         105
2

In [12]:
# city_pairs = users_in_both.select([col("city1"), col("city2")])

# users_in_either = (valid_user_and_city.alias("df1")
#                    .join(city_pairs.alias("cp"), (col("df1.city") == col("cp.city1")))
#                    .join(valid_user_and_city.alias("df2"), col("df2.city") == col("cp.city2"))
#                    .join(user.select("user_id").alias("u"), (col("u.user_id") == col("df1.user_id")) | (col("u.user_id") == col("df2.user_id")))
#                    .groupby([col("df1.city").alias("city1"), col("df2.city").alias("city2")]).count()
#                    .select([col("city1"), col("city2"), col("count").alias("users_in_either")]).where("users_in_either >= 50"))
# print(users_in_either.toPandas())



users_in_either = (city_users.alias("cu1")
                   .join(users_in_both.alias("b"), (col("cu1.city") == col("b.city1")))
                   .join(city_users.alias("cu2"), (col("cu2.city") == col("b.city2")))
                   .select([col("b.city1"), col("b.city2"), 
                            (col("cu1.city_users") + col("cu1.city_users") - col("b.users_in_both")).alias("users_in_either")]))

# print(users_in_either.head(10))

In [13]:
city_sims = (users_in_both.alias("b").join(users_in_either.alias("e"), (col("b.city1") == col("e.city1")) & (col("b.city2") == col("e.city2")))
             .select([col("b.city1"), col("b.city2"), (col("b.users_in_both") / col("e.users_in_either")).alias("jaccard")])
             .orderBy(["jaccard", "city1", "city2"], ascending=[0, 1, 1]))

city_sims_df = city_sims.toPandas()
print(city_sims_df)

                    city1               city2   jaccard
0               Brentwood          Pittsburgh  1.000000
1          Castle Shannon          Pittsburgh  1.000000
2                 Crafton          Pittsburgh  1.000000
3                Emsworth          Pittsburgh  1.000000
4              Enterprise           Las Vegas  1.000000
5               Gerlingen           Stuttgart  1.000000
6              Green Tree          Pittsburgh  1.000000
7              Heidelberg          Pittsburgh  1.000000
8                  Kernen           Stuttgart  1.000000
9      Korntal-Münchingen           Stuttgart  1.000000
10    McCandless Township          Pittsburgh  1.000000
11             Mont-Royal            Montréal  1.000000
12         Montreal-Ouest            Montréal  1.000000
13           McKees Rocks          Pittsburgh  0.974684
14           Blue Diamond           Las Vegas  0.960784
15              Fitchburg             Madison  0.958848
16              East York             Toronto  0

In [19]:
city_sims_data = Data([Histogram(x=city_sims_df["jaccard"])])
py.iplot(city_sims_data)

In [31]:
users_in_both_doubled = (valid_user_and_city.alias("df1").join(valid_user_and_city.alias("df2"), col("df1.city") != col("df2.city"))
                         .join(user.select("user_id").alias("u"), (col("u.user_id") == col("df1.user_id")) & (col("u.user_id") == col("df2.user_id")))
                         .groupby([col("df1.city").alias("city1"), col("df2.city").alias("city2")]).count()
                         .select([col("city1"), col("city2"), col("count").alias("users_in_both")]).where("users_in_both >= 10"))


top_sim_fracs = (users_in_both_doubled.alias("b").join(city_users.alias("cu"), col("b.city1") == col("cu.city"))
                 .select([col("b.city1"), col("b.city2"), (col("b.users_in_both") / col("cu.city_users")).alias("most_sim_frac")])
                 .groupby("city1").agg(max("most_sim_frac").alias("top_most_sim_frac")))  

top_sim_fracs.printSchema()

alt_city_sims = (users_in_both_doubled.alias("b").join(city_users.alias("cu"), col("b.city1") == col("cu.city"))
                 .select([col("b.city1"), col("b.city2"), (col("b.users_in_both") / col("cu.city_users")).alias("most_sim_frac")])
                 .join(top_sim_fracs.alias("tsf"), (col("tsf.city1") == col("b.city1")) & (col("tsf.top_most_sim_frac") == col("most_sim_frac")))
                 .select([col("b.city1").alias("city"), col("b.city2").alias("most_sim_city"), col("tsf.top_most_sim_frac")])
                 .orderBy(["top_most_sim_frac", "city", "most_sim_city"], ascending=[0, 1, 1]))

alt_city_sims.printSchema()
alt_city_sims_df = alt_city_sims.toPandas()
print(alt_city_sims_df)


root
 |-- city1: string (nullable = true)
 |-- top_most_sim_frac: double (nullable = true)

root
 |-- city: string (nullable = true)
 |-- most_sim_city: string (nullable = true)
 |-- top_most_sim_frac: double (nullable = true)

                          city   most_sim_city  top_most_sim_frac
0                    Brentwood      Pittsburgh           1.000000
1               Castle Shannon      Pittsburgh           1.000000
2                      Crafton      Pittsburgh           1.000000
3                     Emsworth      Pittsburgh           1.000000
4                   Enterprise       Las Vegas           1.000000
5                    Gerlingen       Stuttgart           1.000000
6                   Green Tree      Pittsburgh           1.000000
7                   Heidelberg      Pittsburgh           1.000000
8                       Kernen       Stuttgart           1.000000
9           Korntal-Münchingen       Stuttgart           1.000000
10         McCandless Township      Pittsburgh

In [39]:
print(len(alt_city_sims_df.sort("city")["city"]))
print(len(alt_city_sims_df["city"].drop_duplicates()))

349
347


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:1: FutureWarning:

sort(columns=....) is deprecated, use sort_values(by=.....)



In [30]:
alt_city_sims_data = Data([Histogram(x=alt_city_sims_df["top_most_sim_frac"])])
py.iplot(alt_city_sims_data)

In [27]:
completely_related_cities = alt_city_sims.select(["city", "most_sim_city"]).where("top_most_sim_frac == 1.0")

completely_related_cities_df = completely_related_cities.toPandas()
print(completely_related_cities_df)

                   city most_sim_city
0             Brentwood    Pittsburgh
1        Castle Shannon    Pittsburgh
2               Crafton    Pittsburgh
3              Emsworth    Pittsburgh
4            Enterprise     Las Vegas
5             Gerlingen     Stuttgart
6            Green Tree    Pittsburgh
7            Heidelberg    Pittsburgh
8                Kernen     Stuttgart
9    Korntal-Münchingen     Stuttgart
10  McCandless Township    Pittsburgh
11           Mont-Royal      Montréal
12       Montreal-Ouest      Montréal
13        N W Las Vegas     Las Vegas
14           Nellis AFB     Las Vegas
15        Saint Laurent      Montréal
16                Sloan     Las Vegas
17      South Las Vegas     Las Vegas
18    South Queensferry     Edinburgh
19        Spring Valley     Las Vegas
20            Summerlin     Las Vegas
21         Turtle Creek    Pittsburgh
22    Upper Saint Clair    Pittsburgh
23           Weddington     Charlotte
24       West Las Vegas     Las Vegas
25          

In [ ]:
# TODO: find a good rate which "cities" are pretty much the same as other cities and then replace in business data?